# Optimizing_Save_and_Load_Model

在之前的学习中，已经涵盖的内容包括：
- Tensor：Tensor的组成，运算等
- 自动梯度计算：这一机制是梯度下降法训练模型的基础。
- 利用 `torch.nn` 构建模型：通过该模块的神经网络层和函数来搭建深度学习架构。
- Dataset 和 DataLoader：了解这两个概念如何简化数据加载和预处理流程，使数据可以高效地送入模型进行训练。
- TensorBoard 可视化：使用此工具监控和展示训练过程中的各种指标。

接下来这段视频将介绍的新工具：

- **损失函数详解**：探讨不同的损失函数及它们各自的适用场景。
- **PyTorch优化器**：学习多种优化算法，这些算法用于依据损失函数的结果更新模型参数。
- **保存和加载模型**：学习如何保存模型和加载模型

最终，我们将看到：

- **PyTorch 完整训练循环演示**：将上述所有概念整合，实现一个实际运行的模型训练过程。

## 一、Loss Function
下面介绍下pytorch中常用的损失函数，通过[损失函数官方文档介绍](https://pytorch.org/docs/stable/nn.html#loss-functions)可以查看

在深度学习中，损失函数（或称为目标函数）是用来衡量模型输出与目标之间差异的关键组件。PyTorch 提供了多种基础损失函数，以下是一些常用的损失函数及其简要介绍：

### 1. `nn.MSELoss`
- **全称**: Mean Squared Error Loss（均方误差损失）
- **用途**: 回归问题
- **公式**: 
  $
  \text{MSE} = \frac{1}{n} \sum_{i=1}^n (\hat{y}_i - y_i)^2
  $
- **示例**:

In [5]:
import torch
import torch.nn as nn

loss_fn = nn.MSELoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
loss = loss_fn(input, target)
print(loss)

tensor(2.4353, grad_fn=<MseLossBackward0>)


### 2. `nn.CrossEntropyLoss`
- **全称**: Cross Entropy Loss（交叉熵损失）
- **用途**: 多分类问题
- **公式**: 
  $
  \text{CrossEntropy} = -\sum_{i=1}^n y_i \log(\hat{y}_i)
  $
- **示例**:

In [13]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
print(output)

# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
output = loss(input, target)
print(output)

tensor(1.5621, grad_fn=<NllLossBackward0>)
tensor(2.0474, grad_fn=<DivBackward1>)


### 3. `nn.BCELoss`

- **全称**: Binary Cross Entropy Loss（二元交叉熵损失）
- **用途**: 二分类问题
- **公式**: 
  $
  \text{BCE} = -\frac{1}{n} \sum_{i=1}^n [y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i)]
  $
- **示例**:

In [ ]:
loss_fn = nn.BCELoss()
input = torch.sigmoid(torch.randn(3, requires_grad=True))
target = torch.empty(3).random_(2)  # 二元标签
loss = loss_fn(input, target)
print(loss)

### 4. `nn.BCEWithLogitsLoss`
- **全称**: Binary Cross Entropy Loss with Logits（二元交叉熵损失带Logits）
- **用途**: 二分类问题，结合了`Sigmoid`和`BCELoss`
- **公式**: 与BCELoss相同，但输入不需要先通过`Sigmoid`
- **示例**:

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
input = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)  # 二元标签
loss = loss_fn(input, target)
print(loss)


### 5. `nn.NLLLoss`
- **全称**: Negative Log Likelihood Loss（负对数似然损失）
- **用途**: 多分类问题，与`LogSoftmax`配合使用
- **公式**: 
  $
  \text{NLL} = -\sum_{i=1}^n \log(\hat{y}_{i, y_i})
  $
- **示例**:

In [8]:
loss_fn = nn.NLLLoss()
input = torch.randn(3, 5, requires_grad=True)
input = torch.nn.functional.log_softmax(input, dim=1)
target = torch.tensor([1, 0, 4])  # 类别标签
loss = loss_fn(input, target)
print(loss)

tensor(2.5719, grad_fn=<NllLossBackward0>)


### 6. `nn.L1Loss`
- **全称**: Mean Absolute Error Loss（平均绝对误差损失）
- **用途**: 回归问题
- **公式**: 
  $
  \text{L1} = \frac{1}{n} \sum_{i=1}^n |\hat{y}_i - y_i|
  $
- **示例**:

In [9]:
loss_fn = nn.L1Loss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
loss = loss_fn(input, target)
print(loss)

tensor(1.0285, grad_fn=<MeanBackward0>)



### 7. `nn.SmoothL1Loss`
- **全称**: Smooth L1 Loss（平滑L1损失）
- **用途**: 回归问题，结合了`L1`和`L2`损失的优点
- **公式**: 类似于`L1Loss`，在误差较小时使用`L2`，误差较大时使用`L1`
- **示例**:

In [10]:
loss_fn = nn.SmoothL1Loss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
loss = loss_fn(input, target)
print(loss)

tensor(0.5512, grad_fn=<SmoothL1LossBackward0>)



### 8. `nn.KLDivLoss`
- **全称**: Kullback-Leibler Divergence Loss（KL散度损失）
- **用途**: 衡量两个概率分布的差异，通常用于生成模型
- **公式**: 
  $
  \text{KL} = \sum_{i=1}^n y_i (\log(y_i) - \log(\hat{y}_i))
  $
- **示例**:

In [11]:
loss_fn = nn.KLDivLoss()
input = torch.randn(3, 5, requires_grad=True)
input = torch.nn.functional.log_softmax(input, dim=1)
target = torch.randn(3, 5)
target = torch.nn.functional.softmax(target, dim=1)
loss = loss_fn(input, target)
print(loss)

tensor(0.2344, grad_fn=<MeanBackward0>)


D:\Software\anaconda3\envs\jupyter\lib\site-packages\torch\nn\functional.py:2976: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


下面是一个关于交叉熵损失和KL散度的一些理解：

[为什么交叉熵（cross-entropy）可以用于计算代价？](https://www.zhihu.com/question/65288314/answer/244557337)

[交叉熵损失函数（Cross Entropy Loss）](https://zhuanlan.zhihu.com/p/638725320)


**注意：**

实际上，损失函数是一个模型中最核心的部分，一般来说是根据我们自己的任务自行设计损失函数，不会用官方提供的。损失函数设计也是最难的一部分

## 二、Optimizer

Optimizer的主要职责是更新模型的参数（权重和偏置）以最小化损失函数。具体来说，优化器会使用梯度（通过反向传播计算得到）来调整参数。不同的优化算法有不同的更新规则和参数调整策略，例如 SGD（随机梯度下降）、Adam、RMSprop 等。

下面介绍下有关`torch.optim.Optimizer`相关的知识和pytorch中常用的优化器，通过[优化器官方文档介绍](https://pytorch.org/docs/stable/optim.html)可以查看。

### 2.1 torch.optim.Optimizer
`CLASS torch.optim.Optimizer(params, defaults)`

`torch.optim.Optimizer` 是 PyTorch 提供的优化器的基类。它包含多个参数和功能函数，用于管理和更新模型参数。以下是一些关键的参数和功能函数：

**1.关键参数**
- params：需要优化的参数（通常是模型的参数）
- defaults： 包含优化器默认配置的字典，如学习率（lr）、动量（momentum）等,也就是params后的所有参数会打包成**defaults字典**。如`SGD`优化器的`defaults`参数：
```python
class SGD(Optimizer):
    def __init__(self, params, lr=0.01, momentum=0, dampening=0,
                 weight_decay=0, nesterov=False):
        # 验证参数
        if lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if momentum < 0.0:
            raise ValueError("Invalid momentum value: {}".format(momentum))
        if weight_decay < 0.0:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))
        if nesterov and (momentum <= 0 or dampening != 0):
            raise ValueError("Nesterov momentum requires a momentum and zero dampening")
        
        # 将所有参数存储在 defaults 中
        defaults = dict(lr=lr, momentum=momentum, dampening=dampening,
                        weight_decay=weight_decay, nesterov=nesterov)
        super(SGD, self).__init__(params, defaults)
```

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim

# 定义一个简单的线性模型
model = nn.Linear(10, 1)


# 创建一个 SGD 优化器
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.01)

# 打印优化器的 defaults
print(optimizer.defaults)


{'lr': 0.01, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0.01, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'fused': None}


**2.主要功能函数**
- `__init__(self, params, defaults)`: 初始化优化器。params 是需要优化的参数，defaults 是包含默认配置的字典。
+ `zero_grad(self, set_to_none=False)`: 将所有优化的参数的梯度清零。通常在每次反向传播之前调用，以避免累积梯度。set_to_none：如果为 True，则将梯度设置为 None，而不是零。这样做可以稍微提高性能。
- `step(self, closure=None)`: 执行单步优化（参数更新）。这是用户在每个训练步骤中调用的主要函数。`closure`：一个可调用的函数，用于重新计算模型并返回损失。在某些优化算法（如 L-BFGS）中是必需的。
+ `add_param_group(self, param_group)`: 向优化器添加参数组。参数组是一组具有相同优化设置的参数。`param_group`：一个字典，包含参数和它们的优化设置。
```python
param_group = {'params': model.layer.parameters(), 'lr': 1e-3}
optimizer.add_param_group(param_group)
```
一个优化器可以包含多个参数组`param_group`的，**后面添加的参数组如果未指定参数优化设置（如lr），则默认使用`defaults`的参数，而且两个参数组中的参数是不能相同的**


In [19]:
model1 = nn.Linear(5, 1)
# 定义一个新的参数组
new_param_group = {'params': model1.parameters(), 'lr': 0.001}

# 添加新的参数组到优化器中
optimizer.add_param_group(new_param_group)

# 打印所有参数组
for param_group in optimizer.param_groups:
    print(param_group)

{'params': [Parameter containing:
tensor([[ 0.1144, -0.0594,  0.0309,  0.0003, -0.1368,  0.2984, -0.0255,  0.3095,
          0.0515,  0.1672]], requires_grad=True), Parameter containing:
tensor([-0.1795], requires_grad=True)], 'lr': 0.01, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0.01, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'fused': None}
{'params': [Parameter containing:
tensor([[ 0.1362,  0.2245, -0.1155, -0.0211,  0.0911,  0.2797, -0.0533, -0.1589,
          0.1296,  0.0926]], requires_grad=True), Parameter containing:
tensor([0.0492], requires_grad=True)], 'lr': 0.001, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0.01, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'fused': None}


- `state_dict(self)`: 返回优化器的状态字典。**通常用于保存优化器状态，以便后续恢复训练。**
```python
torch.save(optimizer.state_dict(), 'optimizer.pth')
```
+ `load_state_dict(self, state_dict)`:**加载优化器的状态字典。通常用于恢复训练。**
```python
optimizer.load_state_dict(torch.load('optimizer.pth'))
```


### 2.2 常见的优化器

**1.SGD (随机梯度下降, Stochastic Gradient Descent)**

`torch.optim.SGD(params, lr=0.001, momentum=0, dampening=0, weight_decay=0)`，列举出常用的参数：
- `params`: 要优化的参数，可以是一个iterable或者是定义了参数组的字典。**一般是model.parameters()，返回一个生成器。**
- `lr `(学习率): 控制每次参数更新的步长。默认值为0.001。
- `momentum`: 动量因子，有助于加速收敛和减少振荡。默认值为0。**动量因子在梯度下降中引入了物理中的动量概念，通过考虑之前梯度的方向和大小来加速收敛并减少振荡。**
- `dampening`: 动量的抑制因子。默认值为0。
- `weight_decay`: 权重衰减（L2惩罚）。默认值为0。



**2. Adam(Adaptive Moment Estimation)**

`torch.optim.Adam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-8, weight_decay=0)`，列举出常用的参数：
- `params`: 要优化的参数，可以是一个iterable或者是定义了参数组的字典。
- `lr `(学习率): 控制每次参数更新的步长。默认值为0.001。
- `betas`: 用于计算一阶和二阶矩估计的系数。默认值为(0.9, 0.999)。
- `eps (epsilon)`: 防止除零错误的小数。默认值为1e-8。
- `weight_decay`: 权重衰减（L2惩罚）。默认值为0。 

Adam（Adaptive Moment Estimation）优化算法结合了动量（Momentum）和RMSProp的优点，通过自适应调整每个参数的学习率来进行优化。以下是Adam的更新公式：

$ m_t = \beta_1 m_{t-1} + (1 - \beta_1) g_t $

$ v_t = \beta_2 v_{t-1} + (1 - \beta_2) g_t^2 $

$ \hat{m}_t = \frac{m_t}{1 - \beta_1^t} $

$ \hat{v}_t = \frac{v_t}{1 - \beta_2^t} $

$ \theta_{t+1} = \theta_t - \eta \frac{\hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon} $

其中：
- $ m_t $ 是一阶矩估计（动量），对应于梯度的指数加权移动平均。
- $ v_t $ 是二阶矩估计（方差），对应于梯度平方的指数加权移动平均。
- $ \beta_1 $ 和 $ \beta_2 $ 分别控制一阶和二阶矩估计的指数衰减率。
- $ g_t $ 是当前梯度。
- $ \eta $ 是学习率。
- $ \epsilon $ 是一个小数，用于防止除零错误。

**`betas` 参数的意义**

- **`beta1` (默认值0.9)**:
  - 控制一阶矩估计（动量项） \( m_t \) 的指数衰减率。
  - 较大的 `beta1` 值（接近1）意味着动量项更依赖于过去的梯度信息，平滑效果更明显，但反应速度稍慢。
  - 较小的 `beta1` 值意味着动量项更依赖于当前梯度，反应更迅速，但平滑效果减弱。
  - 通常设为0.9，可以在梯度更新中获得较好的平滑效果，避免震荡。

- **`beta2` (默认值0.999)**:
  - 控制二阶矩估计（方差项） \( v_t \) 的指数衰减率。
  - 较大的 `beta2` 值（接近1）意味着方差估计更依赖于过去的梯度平方信息，平滑效果更明显。
  - 较小的 `beta2` 值意味着方差估计更依赖于当前的梯度平方，反应更迅速，但平滑效果减弱。
  - 通常设为0.999，能够更稳定地调整学习率，因为梯度平方的变化通常较慢。

更多关于优化器算法的知识请看(https://arxiv.org/pdf/1609.04747.pdf)

### 2.3 动态调整学习率（torch.optim.lr_scheduler）

PyTorch提供了一些工具来根据训练的进度（例如训练的轮数）动态调整学习率。具体地，`torch.optim.lr_scheduler` 模块中包含了几种不同的方法，其中 `torch.optim.lr_scheduler.ReduceLROnPlateau` 可以根据验证集的表现动态调整学习率。

In [27]:
import torch
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.nn as nn

# 定义一个简单的模型
model = nn.Linear(10, 1)

# 定义损失函数
criterion = nn.MSELoss()

# 定义优化器
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 定义学习率调度器
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

# 或者使用 ReduceLROnPlateau 调度器
# scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

# 假设我们有一个数据加载器，提供批量数据
data_loader = [(torch.randn(32, 10), torch.randn(32, 1)) for _ in range(100)]  # 示例数据

# 训练循环
for epoch in range(5):
    for batch_inputs, batch_targets in data_loader:
        optimizer.zero_grad()   # 清除梯度
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_targets)
        loss.backward()  # 反向传播计算梯度
        optimizer.step()  # 更新参数

    # 在每个epoch结束时更新学习率
    scheduler.step()

    # 如果使用 ReduceLROnPlateau 调度器
    # scheduler.step(loss)

    print(f'Epoch {epoch+1}, Loss: {loss.item()}, Learning Rate: {optimizer.param_groups[0]["lr"]}')


Epoch 1, Loss: 0.9708653688430786, Learning Rate: 0.001
Epoch 2, Loss: 0.9710237383842468, Learning Rate: 0.0001
Epoch 3, Loss: 0.9710696339607239, Learning Rate: 1e-05
Epoch 4, Loss: 0.9710732102394104, Learning Rate: 1.0000000000000002e-06
Epoch 5, Loss: 0.9710735082626343, Learning Rate: 1.0000000000000002e-07


**1. StepLR**

**场景**：
适用于当训练过程中的每隔多少阶段学习率需要按固定比例降低时。

**适用情况**：
通常在固定的epoch间隔后学习率需要调整，如常见的初始快速下降，然后逐渐减缓的训练策略。

**参数**：
- `optimizer`: 优化器实例。
- `step_size`: 每隔多少个epoch调整一次学习率。
- `gamma`: 学习率的衰减因子。

```python
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
```

**2. MultiStepLR**

**场景**：
适用于需要在特定的多个时间点调整学习率的场景。

**适用情况**：
通常用于已经确定好需要在哪些epoch进行学习率调整的训练策略。例如，当模型在某些训练阶段的学习速度减缓时，你可以选择在这些时刻降低学习率，以帮助模型在后期更细致地调整参数。

**参数**：
- `optimizer`: 优化器实例。
- `milestones`: 一个包含若干epoch值的列表，在这些epoch进行学习率调整。
- `gamma`: 学习率的衰减因子。


```python
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[30, 80], gamma=0.1)
```

**3. ExponentialLR**

**场景**：
适用于学习率需要持续以指数方式衰减的场景。

**适用情况**：
常用于训练中希望学习率以平滑的方式逐渐减小的情况，以提高模型的训练稳定性。

**参数**：
- `optimizer`: 优化器实例。
- `gamma`: 每个epoch学习率的衰减因子。

```python
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
```

**4. ReduceLROnPlateau**

**场景**：
适用于需要根据验证集性能动态调整学习率的场景。

**适用情况**：
常用于监控验证损失，当验证损失不再下降时减小学习率，以避免过拟合并改善模型的训练效果。

**参数**：
- `optimizer`: 优化器实例。
- `mode`: 'min' 或 'max'。根据验证集指标的最小值（'min'）或最大值（'max'）来调整学习率。
- `factor`: 学习率的衰减因子。学习率将乘以 `factor`。
- `patience`: 在验证集指标不再提升时，等待多少个epoch再调整学习率。
- `verbose`: 是否输出学习率调整的信息（默认为 `False`）。


```python
import torch
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

# 定义模型
model = ...  # 替换为你的模型
criterion = torch.nn.CrossEntropyLoss()  # 替换为你的损失函数

# 定义优化器
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 定义 ReduceLROnPlateau 调度器
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)

# 假设我们有一个数据加载器，提供批量数据
data_loader = [(torch.randn(32, 10), torch.randint(0, 2, (32,))) for _ in range(100)]  # 示例数据

# 训练循环
for epoch in range(100):
    model.train()
    for batch_inputs, batch_targets in data_loader:
        optimizer.zero_grad()
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_targets)
        loss.backward()
        optimizer.step()
    
    # 假设计算验证损失
    model.eval()
    val_loss = calculate_validation_loss()  # 需要实现的验证损失计算函数

    # 更新学习率
    scheduler.step(val_loss)
    
    print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {val_loss}, Learning Rate: {optimizer.param_groups[0]["lr"]}')

def calculate_validation_loss():
    # 实现你的验证损失计算逻辑
    return torch.tensor(0.1)  # 示例返回值
```

5.CosineAnnealingLR

场景：
适用于需要在训练过程中周期性地调整学习率的场景。

适用情况：
适用于训练周期固定、希望学习率在每个周期内从大到小再回升的情况。`CosineAnnealingLR` 调度器可以帮助模型在训练中保持多样化的学习率变化，通常与重启策略（如SGDR）结合使用，以更好地探索参数空间和提高训练效果。

参数：
- `optimizer`: 优化器实例。
- `T_max`: 学习率退火周期的最大值。表示学习率将在 `T_max` 个epoch内从初始值衰减到最小值，然后再回升。

```python
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)
```

6.CyclicLR

场景：
适用于需要在两个边界之间周期性地变化学习率的场景。

适用情况：
适用于希望在训练过程中学习率在预定范围内不断变化的情况。`CyclicLR` 调度器通过在训练期间周期性地调整学习率，能够增强模型的泛化能力并加速收敛。

参数：
- `optimizer`: 优化器实例。
- `base_lr`: 最小学习率，即周期中的低点。
- `max_lr`: 最大学习率，即周期中的高点。
- `step_size_up`: 增加学习率的步长，即从 `base_lr` 到 `max_lr` 的周期长度。
- `step_size_down` (可选): 减少学习率的步长，即从 `max_lr` 回到 `base_lr` 的周期长度。如果未指定，则默认为 `step_size_up` 的值。
- `mode` (可选): 周期变化模式，默认为 'triangular'，还可以选择 'triangular2' 和 'exp_range'。
- `cycle_momentum` (可选): 是否在周期中调整动量（默认为 `False`）。
- `base_momentum` (可选): 基本动量值。
- `max_momentum` (可选): 最大动量值。

```python
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.01, max_lr=0.1)
data_loader = torch.utils.data.DataLoader(...)
for epoch in range(10):
    for batch in data_loader:
        train_batch(...)
        scheduler.step()
```
7.OneCycleLR

场景：
适用于一个训练周期内需要先增加学习率再减少学习率的情况，通常用于提升模型性能和训练效率。

适用情况：
`OneCycleLR` 是一种周期性学习率调度策略，适用于希望在一个训练周期内快速找到合适的学习率并进行训练的情况。该调度器通过在训练的前半部分逐步增加学习率，后半部分逐步减少学习率，帮助模型达到更好的训练效果和收敛速度。

参数：
- `optimizer`: 优化器实例。
- `max_lr`: 一个周期内的最大学习率。
- `total_steps` 或 `steps_per_epoch` 和 `epochs`:
  - `total_steps`: 总训练步数。使用此参数时，`steps_per_epoch` 和 `epochs` 参数将被忽略。
  - `steps_per_epoch`: 每个epoch中的训练步数。
  - `epochs`: 总训练周期数。
- `pct_start` (可选): 学习率增加到 `max_lr` 的周期比例，默认值为0.3。
- `anneal_strategy` (可选): 学习率衰减策略，默认为 'cos'，也可以设置为 'linear'。
- `div_factor` (可选): 最大学习率与最小学习率的比例因子，默认为 25.0。
- `final_div_factor` (可选): 最终学习率与最大学习率的比例因子，默认为 1e4。
- `cycle_momentum` (可选): 是否在周期中调整动量（默认为 `True`）。
- `base_momentum` (可选): 基本动量值（默认为 0.85）。
- `max_momentum` (可选): 最大动量值（默认为 0.95）。

```python
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=100, epochs=10)

# 训练循环
for epoch in range(10):
    model.train()
    for batch_inputs, batch_targets in data_loader:
        optimizer.zero_grad()
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_targets)
        loss.backward()
        optimizer.step()
        
        # 更新学习率
        scheduler.step()
```

### 2.4 Pytorch_Warmup（结合torch.optim.lr_scheduler一起使用）

[pytorch_warmup](https://github.com/Tony-Y/pytorch_warmup)是一个用于PyTorch的学习率调度库。它的主要作用是通过在训练的早期阶段逐渐增加学习率，帮助模型更稳定地收敛。这种技术被称为"学习率预热"（learning rate warmup）。

**1.作用：**
- 稳定训练：在训练的初期，模型参数通常是随机初始化的，可能会导致梯度不稳定。通过使用较小的学习率，模型能够更稳定地更新参数，避免出现梯度爆炸或梯度消失的问题。
- 加快收敛速度：在预热阶段之后，逐渐增加学习率可以帮助模型快速找到更优的参数，从而加快整体的收敛速度。
- 提高模型性能：在某些情况下，特别是使用大规模数据和复杂模型时，学习率预热可以提高最终模型的性能

**2. base和LinearWarmup源代码：**
```python
import math
from contextlib import contextmanager
from torch.optim import Optimizer


def _check_optimizer(optimizer):
    if not isinstance(optimizer, Optimizer):
        raise TypeError('{} ({}) is not an Optimizer.'.format(
            optimizer, type(optimizer).__name__))


class BaseWarmup(object):
    """Base class for all warmup schedules

    Arguments:
        optimizer (Optimizer): an instance of a subclass of Optimizer
        warmup_params (list): warmup paramters
        last_step (int): The index of last step. (Default: -1)
    """

    def __init__(self, optimizer, warmup_params, last_step=-1):
        ......

    def dampen(self, step=None):
        """Dampen the learning rates.

        Arguments:
            step (int): The index of current step. (Default: None)
        """
        if step is None:
            step = self.last_step + 1
        self.last_step = step

        for group, params in zip(self.optimizer.param_groups, self.warmup_params):
            omega = self.warmup_factor(step, **params)
            group['lr'] *= omega

    @contextmanager
    def dampening(self):
        for group, lr in zip(self.optimizer.param_groups, self.lrs):
            group['lr'] = lr
        yield
        self.lrs = [group['lr'] for group in self.optimizer.param_groups]
        self.dampen()

    def warmup_factor(self, step, **params):
        raise NotImplementedError


def get_warmup_params(warmup_period, group_count):
    ......

class LinearWarmup(BaseWarmup):
    """Linear warmup schedule.

    Arguments:
        optimizer (Optimizer): an instance of a subclass of Optimizer
        warmup_period (int or list): Warmup period
        last_step (int): The index of last step. (Default: -1)
    """

    def __init__(self, optimizer, warmup_period, last_step=-1):
        _check_optimizer(optimizer)
        group_count = len(optimizer.param_groups)
        warmup_params = get_warmup_params(warmup_period, group_count)
        super(LinearWarmup, self).__init__(optimizer, warmup_params, last_step)

    def warmup_factor(self, step, warmup_period):
        return min(1.0, (step+1) / warmup_period)
......
```
**3.使用方法之一：在预热完成后执行原有的lr_scheduler和执行逻辑**

示例：

```python
warmup_period = 2000
num_steps = len(dataloader) * num_epochs - warmup_period
# 定义优化器
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# 定义lr_schedduler
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_steps)
# 定义warm_scheduler
warmup_scheduler = warmup.LinearWarmup(optimizer, warmup_period)
for epoch in range(1,num_epochs+1):
    for batch in dataloader:
        ...
        optimizer.step()
        with warmup_scheduler.dampening():
            if warmup_scheduler.last_step + 1 >= warmup_period:
                lr_scheduler.step()
```
通过源代码分析，可知整体的执行逻辑：
1. 进入上下文管理器：
    - 通过`self.lrs = [group['lr'] for group in self.optimizer.param_groups]`，使得学习率恢复到初始值 0.01。
2. 在上下文管理器中：
    - 判断是否预热完成，完成执行lr_scheduler.step() 恢复原有的学习率调整策略。
3. 退出上下文管理器：
    - warmup_scheduler.dampen() 调整学习率，假设当前步数为 1000，新的学习率为 0.01 * 1000/2000 = 0.005。

## 三、保存和加载模型

在模型训练完成后，我们往往需要保存模型，毕竟是花费很多时间和精力才得到这个结果。

### 3.1模型保存和加载
模型保存有两种方式

**1.保存参数（推荐）**

仅保存模型的参数（state_dict），不包括模型的结构。适用于大多数情况下，因为模型的结构通常是已知的或容易重建的。
使用`torch.save()`进行保存，需要输入要保存的模型参数和路径两个参数：
```python
# 初始化模型
model = SimpleModel()

# 保存模型的参数
torch.save(model.state_dict(), 'model_params.pth')
```
对应的加载方式如下：
```python
# 初始化模型
model = SimpleModel()

# 加载模型的参数
model.load_state_dict(torch.load('model_params.pth'))
```

**2.保存整个模型**

将保存模型的结构和参数，但在加载时可能会有一些不便，因为它依赖于 PyTorch 的版本和环境。
```python
# 保存整个模型
torch.save(model, 'model_complete.pth')

# 加载整个模型
model = torch.load('model_complete.pth')
```


### 3.2 中断训练再继续
在长时间或计算密集型任务（例如大规模数据处理、复杂模拟、机器学习训练等）中，通过定期保存程序状态（即“检查点”），以便在发生中断或失败时能够从最近的检查点恢复继续执行任务，而不必从头开始。

检查点checkpoint通常需要保存`epoch`、模型参数`model.state_dict()`、优化器参数`optimizer.state_dict()`、`loss`

以下是常用的一个模版：

保存
```python
checkpoint_path = "checkpoint.pth"

for epoch in range(num_epochs):
    ......
    
    # 保存检查点
    if (epoch + 1) % 5 == 0:  # 每5个epoch保存一次
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss.item(),
        }, checkpoint_path)
```
加载
```python
# 恢复检查点
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
start_epoch = checkpoint['epoch'] + 1
loss = checkpoint['loss']


# 继续训练
for epoch in range(start_epoch, num_epochs):
    ......
```

## 四、一整个pytorch训练流程

## 数据集

在本教程中，我们将使用由 TorchVision 提供的 Fashion-MNIST 数据集。我们将应用 `torchvision.transforms.Normalize()` 来对图像像素分布进行零均值化和标准化处理，并下载训练和验证数据子集。

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import pytorch_warmup as warmup
import torch.nn as nn
import torch.nn.functional as F

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime


transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

# 创建训练集和验证集Dataset，采用直接加载已有数据方式
training_set = torchvision.datasets.FashionMNIST('./data', train=True, transform=transform, download=True)
validation_set = torchvision.datasets.FashionMNIST('./data', train=False, transform=transform, download=True)

# 创建训练集和验证集Dataloder
training_loader = torch.utils.data.DataLoader(training_set, batch_size=4, shuffle=True, num_workers=2)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=4, shuffle=False, num_workers=2)

# 创建TensorBoard供可视化使用
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))

print('一共有{0}个批次'.format(len(training_loader)))

一共有15000个批次


## 模型

在本例中使用的模型是LeNet-5

In [2]:
class GarmentClassifier(nn.Module):
    def __init__(self):
        super(GarmentClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        output = self.fc3(x)
        
        return output

model = GarmentClassifier()

## 训练优化
定义损失函数，定义优化器，进行训练

In [3]:
# 损失函数，采用交叉熵损失
loss_fn = torch.nn.CrossEntropyLoss()

# 优化器、学习率调整和预热
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, )
warmup_scheduler = warmup.LinearWarmup(optimizer, 10)

EPOCHS = 50

for epoch in range(EPOCHS):
    model.train()
    running_tloss = 0.0
    running_vloss = 0.0
    
    for i, vdata in enumerate(training_loader):
        tinputs, tlabels = vdata
        toutputs = model(tinputs)
        tloss = loss_fn(toutputs, tlabels)
        
        optimizer.zero_grad()
        tloss.backward()
        optimizer.step()
        running_tloss += tloss.item()
        
    model.eval()
    with torch.no_grad():
        for j, vdata in enumerate(validation_loader):
            vinputs, vlabels = vdata
            voutputs = model(vinputs)
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss.item()   
           
    avg_loss = running_tloss / (len(training_loader))
    avg_vloss = running_vloss / (len(validation_loader))

    with warmup_scheduler.dampening():
        if warmup_scheduler.last_step + 1 >= 9:
            lr_scheduler.step(avg_vloss)
    
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch + 1)
    writer.flush() 
    print('EPOCH {0}:avg_loss is {1} and avg_vloss is {2}'.format(epoch, avg_loss, avg_vloss))
    

EPOCH 0:avg_loss is 1.3268629458775745 and avg_vloss is 0.7421066809244454
EPOCH 1:avg_loss is 0.6260463508836615 and avg_vloss is 0.5558972305336967
EPOCH 2:avg_loss is 0.4924594782317057 and avg_vloss is 0.4669824786595302
EPOCH 3:avg_loss is 0.407467496813304 and avg_vloss is 0.4158334927924152
EPOCH 4:avg_loss is 0.35879518366039653 and avg_vloss is 0.35718720184658304
EPOCH 5:avg_loss is 0.3295985228778266 and avg_vloss is 0.3449697941861348
EPOCH 6:avg_loss is 0.31016796532674346 and avg_vloss is 0.32182825101478085
EPOCH 7:avg_loss is 0.2941968655638056 and avg_vloss is 0.33906078571940745
EPOCH 8:avg_loss is 0.28001735381654813 and avg_vloss is 0.33215123484307313
EPOCH 9:avg_loss is 0.2694252235427812 and avg_vloss is 0.3139313252239832
EPOCH 10:avg_loss is 0.25494758353750957 and avg_vloss is 0.29832589491984907
EPOCH 11:avg_loss is 0.24229037467952394 and avg_vloss is 0.30898909759426424
EPOCH 12:avg_loss is 0.23195806347755116 and avg_vloss is 0.2883595319808872
EPOCH 13:av

## 模型保存

In [ ]:
torch.save(model.state_dict(), 'model_params.pth')